# Piecewise Effects

Model non-linear cost curves like economies of scale for investment decisions.

This notebook covers:

- **PiecewiseEffects**: Non-linear cost functions
- **Economies of scale**: Larger systems cost less per unit
- **Investment optimization**: Finding optimal sizes with realistic costs

## Setup

In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px

import flixopt as fx

fx.CONFIG.notebook()

## The Problem: Non-Linear Investment Costs

Investment costs often exhibit **economies of scale**:

| Storage Size | Specific Cost | Why? |
|-------------|---------------|------|
| 0-100 kWh | 0.20 €/kWh | Small systems: high fixed costs per unit |
| 100-300 kWh | 0.15 €/kWh | Medium systems: moderate economies |
| 300-600 kWh | 0.10 €/kWh | Large systems: bulk discounts, efficiency |

This non-linear relationship affects optimal sizing decisions.

### When to Use This Approach

Use `PiecewiseEffects` when:
- Investment costs **decrease per unit** at larger sizes
- You have **tiered pricing** data from vendors
- Accurate **sizing optimization** is important

## Define the Cost Curves

`PiecewiseEffects` requires two components:
1. **piecewise_origin**: The size/capacity segments
2. **piecewise_shares**: The corresponding cost segments for each effect

In [ ]:
# Simple linear cost for comparison
SIMPLE_INVEST_COST = 0.20  # €/kWh constant

# Piecewise investment costs with economies of scale
piecewise_invest_costs = fx.PiecewiseEffects(
    # Size segments (the "origin" - what we're sizing)
    piecewise_origin=fx.Piecewise(
        [
            fx.Piece(start=0, end=100),  # 0-100 kWh
            fx.Piece(start=100, end=300),  # 100-300 kWh
            fx.Piece(start=300, end=600),  # 300-600 kWh
        ]
    ),
    # Corresponding cost segments for the 'costs' effect
    piecewise_shares={
        'costs': fx.Piecewise(
            [
                fx.Piece(start=0, end=20),  # 0-100 kWh → 0-20€ (0.20 €/kWh)
                fx.Piece(start=20, end=50),  # 100-300 kWh → 20-50€ (0.15 €/kWh)
                fx.Piece(start=50, end=80),  # 300-600 kWh → 50-80€ (0.10 €/kWh)
            ]
        )
    },
)

### Understanding the Cost Curve

| Segment | Size Range | Cost Range | Specific Cost |
|---------|------------|------------|---------------|
| 1 | 0→100 kWh | 0→20€ | 0.20 €/kWh |
| 2 | 100→300 kWh | 20→50€ | 0.15 €/kWh |
| 3 | 300→600 kWh | 50→80€ | 0.10 €/kWh |

A 400 kWh system costs: 20€ (first 100) + 30€ (next 200) + 10€ (last 100) = 60€

## Create the Scenario

We'll model a thermal storage system with time-varying energy prices to create price arbitrage opportunities.

In [ ]:
# 3-day scenario with price arbitrage opportunities
timesteps = pd.date_range('2024-01-22', periods=72, freq='h')

# Daily pattern for demand and prices
daily_demand = np.concatenate(
    [
        np.full(8, 100),  # Night: 100 kW
        np.full(4, 120),  # Morning: 120 kW
        np.full(4, 110),  # Midday: 110 kW
        np.full(4, 130),  # Afternoon: 130 kW
        np.full(4, 105),  # Evening: 105 kW
    ]
)

daily_price = np.concatenate(
    [
        np.full(8, 0.08),  # Night: 0.08 €/kWh (cheap)
        np.full(4, 0.20),  # Morning: 0.20 €/kWh (peak)
        np.full(4, 0.12),  # Midday: 0.12 €/kWh (medium)
        np.full(4, 0.25),  # Afternoon: 0.25 €/kWh (peak)
        np.full(4, 0.10),  # Evening: 0.10 €/kWh (medium)
    ]
)

# Repeat for 3 days
heat_demand = np.tile(daily_demand, 3)
energy_price = np.tile(daily_price, 3)

In [ ]:
# Visualize the scenario
fig = px.line(
    x=timesteps.tolist() * 2,
    y=np.concatenate([heat_demand, energy_price * 500]),  # Scale price for visibility
    color=['Demand [kW]'] * 72 + ['Price [€/kWh ×500]'] * 72,
    title='Demand and Price Profiles (3 days)',
    labels={'x': 'Time', 'y': 'Value', 'color': 'Variable'},
)
fig

## Base System (No Storage)

First, create the base system that both models will build upon.

In [ ]:
# Base flow system - will be copied for both models
fs_base = fx.FlowSystem(timesteps)

fs_base.add_elements(
    # Bus
    fx.Bus('Heat', carrier='heat'),
    # Cost tracking
    fx.Effect('costs', '€', 'Costs', is_standard=True, is_objective=True),
    # Heat source with time-varying prices
    fx.Source('HeatSource', outputs=[fx.Flow('Heat', bus='Heat', size=300, effects_per_flow_hour=energy_price)]),
    # Heat demand
    fx.Sink('HeatSink', inputs=[fx.Flow('Heat', bus='Heat', size=1, fixed_relative_profile=heat_demand)]),
)

## Model 1: Simple Linear Investment Costs

Storage with constant cost per kWh.

In [ ]:
fs_simple = fs_base.copy()

fs_simple.add_elements(
    fx.Storage(
        'ThermalStorage',
        charging=fx.Flow('charge', bus='Heat', size=200),
        discharging=fx.Flow('discharge', bus='Heat', size=200),
        capacity_in_flow_hours=fx.InvestParameters(
            effects_of_investment_per_size=SIMPLE_INVEST_COST,  # €/kWh constant
            minimum_size=0,
            maximum_size=600,
        ),
        initial_charge_state=0,
        eta_charge=0.95,
        eta_discharge=0.95,
        relative_loss_per_hour=0.005,
    ),
)

fs_simple.optimize(fx.solvers.HighsSolver())

## Model 2: Piecewise Investment Costs (Economies of Scale)

Storage with decreasing cost per kWh at larger sizes.

In [ ]:
fs_piecewise = fs_base.copy()

fs_piecewise.add_elements(
    fx.Storage(
        'ThermalStorage',
        charging=fx.Flow('charge', bus='Heat', size=200),
        discharging=fx.Flow('discharge', bus='Heat', size=200),
        capacity_in_flow_hours=fx.InvestParameters(
            piecewise_effects_of_investment=piecewise_invest_costs,  # Economies of scale
            minimum_size=0,
            maximum_size=600,
        ),
        initial_charge_state=0,
        eta_charge=0.95,
        eta_discharge=0.95,
        relative_loss_per_hour=0.005,
    ),
)

fs_piecewise.optimize(fx.solvers.HighsSolver())

## Visualize the Investment Cost Curve

In [ ]:
# Plot the piecewise investment cost curve
fs_piecewise.components['ThermalStorage'].capacity_in_flow_hours.piecewise_effects_of_investment.plot(
    title='Storage Investment Costs (Economies of Scale)'
)

## Compare Results

In [ ]:
# Extract results
simple_size = fs_simple.solution['ThermalStorage|size'].item()
simple_cost = fs_simple.solution['costs'].item()

piecewise_size = fs_piecewise.solution['ThermalStorage|size'].item()
piecewise_cost = fs_piecewise.solution['costs'].item()

print('=== Optimization Results ===')
print('\nSimple model (0.20 €/kWh constant):')
print(f'  Optimal size: {simple_size:.0f} kWh')
print(f'  Total cost:   {simple_cost:.2f} €')

print('\nPiecewise model (economies of scale):')
print(f'  Optimal size: {piecewise_size:.0f} kWh')
print(f'  Total cost:   {piecewise_cost:.2f} €')

print('\nDifference:')
print(f'  Size: {piecewise_size - simple_size:+.0f} kWh ({(piecewise_size - simple_size) / simple_size * 100:+.1f}%)')
print(f'  Cost: {piecewise_cost - simple_cost:+.2f} € ({(piecewise_cost - simple_cost) / simple_cost * 100:+.1f}%)')

The piecewise model typically results in a **larger optimal size** because larger systems are proportionally cheaper, making additional capacity more attractive.

## Analyze Storage Operation

In [ ]:
# Storage operation heatmap
fs_piecewise.statistics.plot.heatmap('ThermalStorage')

In [ ]:
# Energy flow visualization
fs_piecewise.statistics.plot.sankey.flows()

In [ ]:
# Heat bus balance
fs_piecewise.statistics.plot.balance('Heat')

## Key Concepts

### PiecewiseEffects Syntax

```python
fx.PiecewiseEffects(
    # Size/capacity segments
    piecewise_origin=fx.Piecewise([
        fx.Piece(start=size_min_1, end=size_max_1),
        fx.Piece(start=size_min_2, end=size_max_2),
    ]),
    # Corresponding cost segments for each effect
    piecewise_shares={
        'effect_name': fx.Piecewise([
            fx.Piece(start=cost_min_1, end=cost_max_1),
            fx.Piece(start=cost_min_2, end=cost_max_2),
        ]),
    },
)
```

### Multiple Effects

You can define piecewise costs for multiple effects (e.g., costs AND CO2):

```python
piecewise_shares={
    'costs': fx.Piecewise([...]),
    'CO2': fx.Piecewise([...]),
}
```

### Where to Use PiecewiseEffects

| Parameter | Location | Use Case |
|-----------|----------|----------|
| `piecewise_effects_of_investment` | `InvestParameters` | Economies of scale in sizing |
| `piecewise_effects` | `Flow` | Non-linear operational costs |

## Advanced: Combining with Other Investment Parameters

`PiecewiseEffects` can be combined with other investment parameters:

In [ ]:
# Example: Piecewise costs + fixed investment cost + non-mandatory investment
advanced_invest = fx.InvestParameters(
    # Fixed cost (connection fee, permits, etc.)
    effects_of_investment={'costs': 50},  # 50€ fixed if built
    # Piecewise variable costs (economies of scale)
    piecewise_effects_of_investment=piecewise_invest_costs,
    # Size constraints
    minimum_size=50,  # Must be at least 50 kWh if built
    maximum_size=600,
    # Not mandatory - can choose not to build at all (this is the default)
    mandatory=False,
)

print('Advanced investment parameters configured')
print('  Fixed cost: 50€')
print('  Variable cost: piecewise (0.20 → 0.15 → 0.10 €/kWh)')
print('  Size range: 50-600 kWh (or 0 if not built)')

## Summary

You learned how to:

- Define **piecewise investment cost curves** using `PiecewiseEffects`
- Model **economies of scale** for realistic sizing decisions
- **Compare** linear vs piecewise cost impacts on optimal sizing
- **Combine** piecewise costs with other investment parameters

### When to Use This vs Other Approaches

| Approach | Use When | Example |
|----------|----------|--------|
| Time-varying factors | Efficiency varies with external conditions | Heat pump COP vs temperature |
| PiecewiseConversion | Efficiency varies with load level | Gas engine efficiency curve |
| **PiecewiseEffects** (this notebook) | Costs vary non-linearly with size | Economies of scale |

### Next Steps

- **[07-scenarios-and-periods](07-scenarios-and-periods.ipynb)**: Multi-scenario planning
- **[08-large-scale-optimization](08-large-scale-optimization.ipynb)**: Computational efficiency techniques